# Gifty SOTA Worker: 10-Dimension Matrix (VLM)

This worker implements the **Gifty 10-Dimensions Matrix** for product scoring.
It uses **Qwen2-VL-7B-Instruct** to perceive products visually and psychometrically.

### The 10 Axes (0.0 - 1.0):
1. **Wow-Factor** (Surprise)
2. **Warmth** (Emotional Temp)
3. **Romance** (Intimacy)
4. **Practicality** (Utility)
5. **Usage Frequency**
6. **Occasion Versatility**
7. **Aesthetics** (Visual Value)
8. **Social Risk**
9. **Age Suitability** (Vector)
10. **Gender Bias** (Vector)

### Output
Generates a structured JSON object stored in `llm_gift_vector`.

In [ ]:
!pip -q install -U transformers accelerate bitsandbytes qwen_vl_utils requests Pillow tqdm

In [ ]:
import os
import json
import re
import logging
import sys
import torch
import requests
from PIL import Image
from io import BytesIO
from kaggle_secrets import UserSecretsClient
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

# Setup
API_BASE_URL = "https://api.giftyai.ru"
DEBUG = True
try:
    user_secrets = UserSecretsClient()
    INTERNAL_TOKEN = user_secrets.get_secret("INTERNAL_API_TOKEN")
except:
    INTERNAL_TOKEN = os.getenv("INTERNAL_API_TOKEN", "default_token")

MODEL_ID = "Qwen/Qwen2-VL-7B-Instruct"
MODEL_VERSION = "v3.0-matrix-10d"

logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s')
logger = logging.getLogger("GiftyMatrixWorker")

In [ ]:
logger.info("Loading Qwen2-VL-7B...")
model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_ID, torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained(MODEL_ID)
model.eval()
logger.info("Model Loaded.")

In [ ]:
SYSTEM_PROMPT = """You are an expert luxury gift buyer and psychologist.
Your task is to evaluate a product across 10 psychological dimensions.
Analyze the image (packaging, quality) and the text triggers.

OUTPUT FORMAT: Return ONLY a valid JSON object. No markdown, no pre-text.
JSON Structure:
{
  "reasoning": "Brief analysis of visual quality, vibe, and target audience (2-3 sentences).",
  "scores": {
    "wow_factor": 0.0-1.0,      // Surprise, delight, uniqueness
    "warmth": 0.0-1.0,          // Cozy, emotional, handcrafted feel vs Cold/Tech
    "romance": 0.0-1.0,         // Intimacy level (High=Partner only, Low=Colleague)
    "practicality": 0.0-1.0,    // Utility vs Decor/Fun
    "usage_frequency": 0.0-1.0, // Daily vs Once/Seasonal
    "occasion_versatility": 0.0-1.0, // Universal (Birthday) vs Niche (Wedding)
    "aesthetics": 0.0-1.0,      // Visual premium feel, packaging quality
    "social_risk": 0.0-1.0,     // Chance of being awkward/useless (High risk = Clothes, Perfume)
    "giftability_total": 0.0-1.0 // Overall suitability as a gift
  },
  "demographics": {
    "gender": "male" | "female" | "unisex",
    "age_min": int,
    "age_max": int
  }
}
"""

def get_image(url):
    try:
        if not url: return None
        resp = requests.get(url, timeout=5)
        return Image.open(BytesIO(resp.content)).convert("RGB")
    except: return None

def extract_json(text):
    try:
        # Find first { and last }
        text = text[text.find('{'):text.rfind('}')+1]
        return json.loads(text)
    except:
        return None

def process_one(item):
    img = get_image(item.get('image_url'))
    
    user_content = [
        {"type": "text", "text": f"Product: {item.get('title')}\nCategory: {item.get('category')}\nPrice: {item.get('price')}"}
    ]
    if img:
        user_content.insert(0, {"type": "image", "image": item['image_url']})
        
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_content}
    ]
    
    text_in = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, _ = process_vision_info(messages)
    
    inputs = processor(
        text=[text_in], images=image_inputs, padding=True, return_tensors="pt"
    ).to(model.device)
    
    # Generate
    generated_ids = model.generate(**inputs, max_new_tokens=512, do_sample=False)
    gen_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]
    
    # Parse Output
    # Remove prompt from output (Qwen sometimes echoes)
    if "OUTPUT FORMAT" in gen_text:
        response_only = gen_text.split("assistant\n")[-1]
    else:
        response_only = gen_text
        
    data = extract_json(response_only)
    
    if not data:
        logger.error(f"Failed to parse JSON for {item.get('gift_id')}. Raw: {response_only[:100]}...")
        return None
        
    return data

In [ ]:
import time

headers = {"X-Internal-Token": INTERNAL_TOKEN}

logger.info("Starting SOTA Loop...")
while True:
    try:
        # 1. Fetch
        resp = requests.get(f"{API_BASE_URL}/internal/scoring/tasks?limit=5", headers=headers)
        tasks = resp.json()
        if not tasks:
            time.sleep(60)
            continue
            
        logger.info(f"Processing batch of {len(tasks)}")
        results = []
        for t in tasks:
            try:
                data = process_one(t)
                if data:
                    res = {
                        "gift_id": t['gift_id'],
                        "llm_gift_score": data['scores'].get('giftability_total', 0.0),
                        "llm_gift_reasoning": data.get('reasoning', ''),
                        "llm_gift_vector": data,
                        "llm_scoring_model": MODEL_ID,
                        "llm_scoring_version": MODEL_VERSION
                    }
                    results.append(res)
            except Exception as e:
                logger.error(f"Error item {t['gift_id']}: {e}")
        
        if results:
            requests.post(f"{API_BASE_URL}/internal/scoring/submit", json={"results": results}, headers=headers)
            logger.info(f"Submitted {len(results)}")
            
    except Exception as e:
        logger.error(f"Loop Error: {e}")
        time.sleep(30)